In [5]:
import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder

## データセットの読み込み

iris の分類は 4 次元の特徴量を使用した 3 値分類のタスク


In [4]:
# データセットの読み込み
iris = sns.load_dataset('iris')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [6]:
# カテゴリ変数を数値に変換
iris.loc[:, 'species'] = iris.loc[:, 'species'].map({'setosa':0, 'versicolor':1, 'virginica':2})
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


PyTorch を使用する場合は、torch.Tensor 型に変換する必要があるので、torch.Tensor 型に変換。

ここで注意すべきことは、ターゲット t の型であり、t の型はタスクによって変える必要がある。(適切な損失関数がタスクによって異なるため。以下()内には代表的な損失関数を記載)

- 回帰の場合: torch.float32(MSE)
- 二値分類の場合: torch.float32(Binary Cross Entropy; BCE)
- 多値分類の場合: torch.int64(Cross Entropy)

今回は 3 値分類のため、torch.int64 とする。


In [7]:
class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame, features: list[str], labels: list[str]):
        self.features = df[features].values
        self.labels = df[labels].values.astype(np.int64)  # 今回は3値分類のためtarget変数はint64
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        feature = torch.FloatTensor(self.features[idx])
        label = torch.LongTensor(self.labels[idx])
        return feature, label

In [11]:
# データセットの作成
iris_dataset = IrisDataset(iris, ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], ['species'])
print(len(iris_dataset))
print(iris_dataset[100])

150
(tensor([6.3000, 3.3000, 6.0000, 2.5000]), tensor([2]))


In [14]:
# Datasetの分割
## train:val:test = 8:1:1
n_train = int(len(iris_dataset) * 0.8)
n_val = int(len(iris_dataset) * 0.1)
n_test = int(len(iris_dataset) * 0.1)

torch.manual_seed(42)
train, val, test = torch.utils.data.random_split(iris_dataset, [n_train, n_val, n_test])
print(len(train), len(val), len(test))

120 15 15


In [15]:
help(torch.utils.data.random_split)

Help on function random_split in module torch.utils.data.dataset:

random_split(dataset: torch.utils.data.dataset.Dataset[~T], lengths: Sequence[Union[int, float]], generator: Optional[torch._C.Generator] = <torch._C.Generator object at 0xffff53a6e050>) -> List[torch.utils.data.dataset.Subset[~T]]
    Randomly split a dataset into non-overlapping new datasets of given lengths.
    
    If a list of fractions that sum up to 1 is given,
    the lengths will be computed automatically as
    floor(frac * len(dataset)) for each fraction provided.
    
    After computing the lengths, if there are any remainders, 1 count will be
    distributed in round-robin fashion to the lengths
    until there are no remainders left.
    
    Optionally fix the generator for reproducible results, e.g.:
    
    Example:
        >>> # xdoctest: +SKIP
        >>> generator1 = torch.Generator().manual_seed(42)
        >>> generator2 = torch.Generator().manual_seed(42)
        >>> random_split(range(10), [3,